In [ ]:
import config
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
from torch.autograd import Function
from tqdm import tqdm
from utility_uad_svm import create_sequences_for_svm, load_data, create_dataloaders
from utility_uad_svm import get_features, complex_balance_data, standardize_features

#加载所有参数 
SRC_IDS = config.SRC_IDS # [1, 2, 3, 4, 5, 6, 7, 8]
TGT_ID = config.TGT_ID    # 0
CSV_PATHS = config.CSV_PATHS # 包含了所有路径的字典
df_src, df_tgt = load_data(SRC_IDS, TGT_ID, CSV_PATHS)
X_src, y_src, S_src, I_src_raw, X_tgt, y_tgt, S_tgt, TGT_INDICES = create_sequences_for_svm(df_src, df_tgt, config)

--- [Config] 正在使用的设备: cpu ---
[load_data] 源域数据形状: (710063, 13), 目标域数据形状: (12614, 14)

--- [序列制作] 正在创建 SVM 所需的序列... ---
  - 正在处理源域 (仿真) 数据...
  - X 形状: (709999, 5, 4), y 形状: (709999,), S 形状: (709999,)

  - 正在处理目标域 (真实) 数据 (无标签处理)...
  - X 形状: (12538, 5, 4), y 形状: (0,), S 形状: (12538,)
--- 序列制作完成 ---


In [2]:
# （在你成功制作序列后执行此代码块）

# --- 步骤 1: 加载训练好的 G_f 特征提取器 ---
print("\n--- [步骤 1] 正在加载 G_f 模型... ---")
from models import LSTMFeatureExtractor # 确保导入了模型定义
from utility_uad_svm import get_features, complex_balance_data, standardize_features
import os
import torch
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
# 确保导入了 NumPy 和 Torch
import numpy as np
import torch
import os


--- [步骤 1] 正在加载 G_f 模型... ---


In [3]:
# F_src 和 F_tgt 是原始的、未缩放的特征向量
device = torch.device(config.DEVICE)
G_f = LSTMFeatureExtractor(
    input_size=config.LSTM_INPUT_SIZE,
    hidden_size=config.LSTM_HIDDEN_SIZE,
    num_layers=config.LSTM_NUM_LAYERS,
    dropout=config.LSTM_DROPOUT
).to(device)

G_f_path = os.path.join(config.MODEL_SAVE_DIR, "G_f_final.pth")
G_f.load_state_dict(torch.load(G_f_path, map_location=device))
G_f.eval()

F_src_raw = get_features(G_f, X_src, config.BATCH_SIZE, device) 
F_tgt_raw = get_features(G_f, X_tgt, config.BATCH_SIZE, device)

  - 正在执行特征提取...


  - 提取特征中: 100%|██████████| 5547/5547 [01:07<00:00, 81.74it/s]


  - 特征提取完毕。输出形状: (709999, 256)
  - 正在执行特征提取...


  - 提取特征中: 100%|██████████| 98/98 [00:01<00:00, 88.71it/s] 

  - 特征提取完毕。输出形状: (12538, 256)


In [4]:
# ----------------------------------------------------------------------
# 在标准化之后，预测之前，强制清洗 NaN
# ----------------------------------------------------------------------

def clean_features(F_feat):
    """
    清除特征中的 NaN 和 Inf 值。
    """
    # 查找并计数 NaN
    nan_count = np.sum(np.isnan(F_feat))
    inf_count = np.sum(np.isinf(F_feat))
    
    if nan_count > 0 or inf_count > 0:
        print(f"【警告】发现 {nan_count} 个 NaN 和 {inf_count} 个 Inf。正在执行清理...")
        
        # 将所有 NaN/Inf 替换为 0 (或其他合适的默认值)
        # 另一种方法是使用特征的均值，但简单替换为 0 更安全，因为特征已经标准化（均值接近 0）
        F_feat = np.nan_to_num(F_feat, nan=0.0, posinf=0.0, neginf=0.0)
        print("清理完成。")
    return F_feat

# ----------------------------------------------------------------------
# ！！在你脚本中 Step 1/2 之后，插入以下调用：
# ----------------------------------------------------------------------

# F_src_scaled, F_tgt_scaled = standardize_features(F_src_raw, F_tgt_raw) 
# ... 标准化代码之后 ...

# 立即清洗特征


In [5]:
print("\n--- [关键修正] 正在执行标准化前数据清洗... ---")

# 1. 强制清洗源域特征 (F_src_raw) - 避免 Scaler 污染
F_src_raw = clean_features(F_src_raw) 
# 2. 强制清洗目标域特征 (F_tgt_raw) - 避免其原始 Inf/NaN 影响 transform 
F_tgt_raw = clean_features(F_tgt_raw) 

# --- [步骤 1] 正在进行特征标准化... ---
# 现在 standardize_features 可以安全运行，因为它接收的是干净的 F_src_raw
F_src_scaled, F_tgt_scaled = standardize_features(F_src_raw, F_tgt_raw) 
# (你的 standardize_features 内部已包含 sigma=0 修复)

print(f" - F_src_scaled 形状: {F_src_scaled.shape}")
print(f" - F_tgt_scaled 形状: {F_tgt_scaled.shape}")



--- [关键修正] 正在执行标准化前数据清洗... ---
【警告】发现 1352448 个 NaN 和 0 个 Inf。正在执行清理...
清理完成。

--- [标准化] 正在对特征进行标准化... ---
  - 特征标准化完成 (基于源域统计量)。
 - F_src_scaled 形状: (709999, 256)
 - F_tgt_scaled 形状: (12538, 256)


In [6]:
# --- 步骤 2: 源域数据分组与精细平衡 ---
print("\n--- [步骤 2] 正在对源域数据进行分组和精细平衡... ---")

# 2a. 按卫星类型分组 (使用缩放后的特征)
idx_src_gps = np.where(S_src == 0)[0] # GPS 卫星的索引
idx_src_pl = np.where(S_src == 1)[0]  # PL 卫星的索引

# 分组缩放后的特征 F_src_scaled
F_src_gps = F_src_scaled[idx_src_gps]
y_src_gps = y_src[idx_src_gps]
S_src_gps = S_src[idx_src_gps] # 传入 GPS 组的 Sat_Type

F_src_pl = F_src_scaled[idx_src_pl]
y_src_pl = y_src[idx_src_pl]
S_src_pl = S_src[idx_src_pl] # 传入 PL 组的 Sat_Type


# 2b. 平衡 GPS 组
print("\n--- 平衡 GPS 组数据 (LOS/NLOS) ---")
# F_gps_bal, y_gps_bal 是平衡后的 GPS 训练数据
F_gps_bal, y_gps_bal = complex_balance_data(F_src_gps, y_src_gps, S_src_gps, random_state=42)

# 2c. 平衡 PL 组
print("\n--- 平衡 PL 组数据 (LOS/NLOS) ---")
# F_pl_bal, y_pl_bal 是平衡后的 PL 训练数据
F_pl_bal, y_pl_bal = complex_balance_data(F_src_pl, y_src_pl, S_src_pl, random_state=42)


# ----------------------------------------------------
# 此时，你得到了最终用于训练的数据：F_gps_bal, y_gps_bal, F_pl_bal, y_pl_bal
# ----------------------------------------------------


--- [步骤 2] 正在对源域数据进行分组和精细平衡... ---

--- 平衡 GPS 组数据 (LOS/NLOS) ---
  - 原始类别分布: Counter({np.int64(1): 343282, np.int64(0): 102860})
✅ 特征空间精细平衡完成。
 - GPS LOS/NLOS 各: 102860 条
 - PL LOS/NLOS 各: 0 条
 - 平衡后总样本数: 205720

--- 平衡 PL 组数据 (LOS/NLOS) ---
  - 原始类别分布: Counter({np.int64(0): 230541, np.int64(1): 33316})
✅ 特征空间精细平衡完成。
 - GPS LOS/NLOS 各: 0 条
 - PL LOS/NLOS 各: 33316 条
 - 平衡后总样本数: 66632


In [7]:
from sklearn.model_selection import train_test_split

# 定义每个专家模型的最大样本数
MAX_SVM_SAMPLES = 20000 # <-- 确保这里是 20000

print("\n--- [步骤 2.5] 对平衡后的数据进行二次欠采样 (控制 SVM 训练规模) ---")

# --- GPS 专家模型二次采样 ---
if len(F_gps_bal) > MAX_SVM_SAMPLES:
    # 修正：我们不使用 train_test_split，而是使用 train_test_split 来获得一个 20k 的子集
    
    # 1. 使用 train_test_split 获得训练集部分（20k）
    F_gps_keep, _, y_gps_keep, _ = train_test_split(
        F_gps_bal, y_gps_bal, 
        train_size=MAX_SVM_SAMPLES, # 返回数组 F_gps_keep 的大小是 20000
        random_state=42,
        stratify=y_gps_bal 
    )
    
    print(f" - GPS 训练集规模已从 {len(F_gps_bal)} 减少到 {len(F_gps_keep)}。")
    F_gps_bal = F_gps_keep
    y_gps_bal = y_gps_keep
else:
    print(f" - GPS 训练集 ({len(F_gps_bal)} 样本) 小于 {MAX_SVM_SAMPLES}，无需采样。")


# --- PL 专家模型二次采样 ---
if len(F_pl_bal) > MAX_SVM_SAMPLES:
    # 1. 使用 train_test_split 获得训练集部分（20k）
    F_pl_keep, _, y_pl_keep, _ = train_test_split(
        F_pl_bal, y_pl_bal, 
        train_size=MAX_SVM_SAMPLES, # 返回数组 F_pl_keep 的大小是 20000
        random_state=42,
        stratify=y_pl_bal 
    )
    
    print(f" - PL 训练集规模已从 {len(F_pl_bal)} 减少到 {len(F_pl_keep)}。")
    F_pl_bal = F_pl_keep
    y_pl_bal = y_pl_keep
else:
    print(f" - PL 训练集 ({len(F_pl_bal)} 样本) 小于 {MAX_SVM_SAMPLES}，无需采样。")


--- [步骤 2.5] 对平衡后的数据进行二次欠采样 (控制 SVM 训练规模) ---
 - GPS 训练集规模已从 205720 减少到 20000。
 - PL 训练集规模已从 66632 减少到 20000。


In [8]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
# 确保在你的脚本顶部导入了 tqdm
from tqdm.auto import tqdm
import numpy as np
import joblib
import os 
from sklearn.svm import LinearSVC # 需要导入新的类
# 假设 svm_gps, svm_pl, F_tgt_scaled, S_tgt, y_tgt, config 等变量已在内存中

# --- 步骤 3: 训练双 SVM 专家模型 ---
print("\n--- [步骤 3] 正在训练 GPS 和 PL 专家模型... ---")

# 3a. 训练 GPS 专家模型
svm_gps = SVC(kernel='rbf', C=100.0, class_weight='balanced', random_state=42)
if len(F_gps_bal) > 0:
    for _ in tqdm(range(1), desc="训练 SVM_GPS 专家模型", ncols=80):
        svm_gps.fit(F_gps_bal, y_gps_bal)
    print(f" - SVM_GPS 训练完成。训练样本数: {len(F_gps_bal)}")
else:
    print(" - 警告：SVM_GPS 样本不足，跳过训练。")

# 3b. 训练 PL 专家模型
svm_pl = SVC(kernel='rbf', C=100.0, class_weight='balanced', random_state=42)
if len(F_pl_bal) > 0:
    for _ in tqdm(range(1), desc="训练 SVM_PL 专家模型", ncols=80):
        svm_pl.fit(F_pl_bal, y_pl_bal)
    print(f" - SVM_PL 训练完成。训练样本数: {len(F_pl_bal)}")
else:
    print(" - 警告：SVM_PL 样本不足，跳过训练。")
# 假设 svm_gps 和 svm_pl 对象已在内存中

# (在 SVM_GPS 和 SVM_PL 训练完成后)

print("\n--- [SVM 训练集性能诊断] ---")

# --- 步骤 6: 保存 SVM 专家模型 ---
print("\n--- [步骤 6] 正在保存 SVM 专家模型... ---")

# 确保保存目录存在
os.makedirs(config.MODEL_SAVE_DIR, exist_ok=True)

# 定义保存路径，使用 config 中已有的 MODEL_SAVE_DIR
gps_path = os.path.join(config.MODEL_SAVE_DIR, "svm_gps_expert.pkl")
pl_path = os.path.join(config.MODEL_SAVE_DIR, "svm_pl_expert.pkl")

# 保存 SVM 模型
joblib.dump(svm_gps, gps_path)
joblib.dump(svm_pl, pl_path)

print(f" - SVM_GPS 专家模型已保存到: {gps_path}")
print(f" - SVM_PL 专家模型已保存到: {pl_path}")









--- [步骤 3] 正在训练 GPS 和 PL 专家模型... ---


训练 SVM_GPS 专家模型:   0%|                              | 0/1 [00:00<?, ?it/s]

 - SVM_GPS 训练完成。训练样本数: 20000


训练 SVM_PL 专家模型:   0%|                               | 0/1 [00:00<?, ?it/s]

 - SVM_PL 训练完成。训练样本数: 20000

--- [SVM 训练集性能诊断] ---

--- [步骤 6] 正在保存 SVM 专家模型... ---
 - SVM_GPS 专家模型已保存到: c:\Users\yangj\Desktop\4JYY\4JYY\Transfer Learning\SVM\checkpoints\svm_gps_expert.pkl
 - SVM_PL 专家模型已保存到: c:\Users\yangj\Desktop\4JYY\4JYY\Transfer Learning\SVM\checkpoints\svm_pl_expert.pkl


In [9]:
import numpy as np
import pandas as pd
import joblib
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC

# --- 假设以下变量和函数已在内存中并已准备好 ---
# F_tgt_scaled: 目标域标准化特征 (来自 standardize_features)
# S_tgt: 目标域卫星类型数组
# y_tgt: 目标域标签数组 (可能是空的占位符)
# config: 配置对象
# df_tgt: 原始目标域 DataFrame
# TGT_INDICES: 原始索引数组 (来自 create_sequences_for_svm)
# ----------------------------------------------------

# --- 警告: 请确保在运行此块代码之前，已加载或训练了 SVM 模型 ---
try:
    # 加载已训练的专家模型 (假设它们在 config.MODEL_SAVE_DIR/checkpoints 路径下)
    MODEL_DIR = config.MODEL_SAVE_DIR
    svm_gps = joblib.load(os.path.join(MODEL_DIR, "svm_gps_expert.pkl"))
    svm_pl  = joblib.load(os.path.join(MODEL_DIR, "svm_pl_expert.pkl"))
    
    # 假设 F_gps_bal 和 F_pl_bal 至少是空数组，以进行安全检查
    F_gps_bal = np.array([]) 
    F_pl_bal = np.array([]) 
    # 在实际训练脚本中，这两个数组会通过 complex_balance_data 赋值。
    # 这里我们只检查长度，避免报错。

except Exception as e:
    print(f"致命错误：无法加载 SVM 模型或初始化 F_bal 数组。请先运行完整的训练脚本。错误: {e}")
    # exit()
# ----------------------------------------------------


# --- 步骤 4: 目标域预测与路由 ---
print("\n--- [步骤 4] 目标域预测 (路由中)... ---")

# 4a. 目标域分组 (路由)
idx_tgt_gps = np.where(S_tgt == 0)[0] 
idx_tgt_pl = np.where(S_tgt == 1)[0] 

F_tgt_gps = F_tgt_scaled[idx_tgt_gps]
F_tgt_pl = F_tgt_scaled[idx_tgt_pl]

# 4b. 预测和重组
y_pred_tgt = np.zeros(len(F_tgt_scaled), dtype=np.int64) 

# GPS 预测 (仅在模型训练和数据存在时执行)
if len(F_gps_bal) > 0 and len(F_tgt_gps) > 0:
    with tqdm(total=len(F_tgt_gps), desc="  - GPS 专家预测中", leave=False) as pbar:
        y_pred_gps = svm_gps.predict(F_tgt_gps)
        y_pred_tgt[idx_tgt_gps] = y_pred_gps
        pbar.update(len(F_tgt_gps))
    print(f" - GPS 目标域预测完成。样本数: {len(F_tgt_gps)}")

# PL 预测
if len(F_pl_bal) > 0 and len(F_tgt_pl) > 0:
    with tqdm(total=len(F_tgt_pl), desc="  - PL 专家预测中", leave=False) as pbar:
        y_pred_pl = svm_pl.predict(F_tgt_pl)
        y_pred_tgt[idx_tgt_pl] = y_pred_pl
        pbar.update(len(F_tgt_pl))
    print(f" - PL 目标域预测完成。样本数: {len(F_tgt_pl)}")

print(" - 预测结果重组完成。")


# --- 步骤 5: 评估和报告 ---
print("\n--- [步骤 5] 评估和报告 ---")

if len(y_tgt) > 0:
    # ... (评估代码) ...
    accuracy = accuracy_score(y_tgt, y_pred_tgt)
    report = classification_report(y_tgt, y_pred_tgt, 
                                   target_names=[f'Class {i}' for i in range(config.NUM_CLASSES)], 
                                   output_dict=False, zero_division=0)
    
    print("\n=======================================================")
    print("✅ DANN-LSTM 双 SVM 专家模型性能报告：")
    print(f"目标域总体 SVM 准确率 (Accuracy): {accuracy:.4f}")
    print("目标域 SVM 评估报告 (GPS/PL 路由后的综合性能):\n", report)
    print("=======================================================")
else:
    print("\n=======================================================")
    print("✅ 预测完成。目标域无标签，跳过评估。")
    print(f"已生成 {len(y_pred_tgt)} 条预测结果 (y_pred_tgt)。")
    print("=======================================================")


# --- 步骤 6: 预测结果回溯合并与保存 ---
print("\n--- [步骤 6] 预测结果回溯合并与保存 ---")

if 'df_tgt' in locals() and 'TGT_INDICES' in locals():
    
    # 1. 创建一个与原始 df_tgt 长度相同的空预测列
    full_predictions = pd.Series(
        data=np.nan, 
        index=df_tgt.index, 
        dtype=float 
    )

    # 2. ！！关键的回溯操作：使用 TGT_INDICES 映射预测值 ！！
    full_predictions.loc[TGT_INDICES] = y_pred_tgt

    # 3. 将新的预测列附加到原始 DataFrame
    df_tgt['predicted_multipath'] = full_predictions

    # 4. 定义输出并保存文件
    OUTPUT_DIR = os.path.join(config.MODEL_SAVE_DIR, "results")
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    OUTPUT_PATH = os.path.join(OUTPUT_DIR, "tgt_data_with_predictions.csv")

    df_tgt.to_csv(OUTPUT_PATH, index=False)

    print(f"✅ 最终结果文件已成功保存到: {OUTPUT_PATH}")
    print("文件已包含所有原始列，并在末尾附加了 'predicted_multipath' 列。")
else:
    print("警告：缺少 df_tgt 或 TGT_INDICES 变量，跳过结果文件保存。")


--- [步骤 4] 目标域预测 (路由中)... ---
 - 预测结果重组完成。

--- [步骤 5] 评估和报告 ---

✅ 预测完成。目标域无标签，跳过评估。
已生成 12538 条预测结果 (y_pred_tgt)。

--- [步骤 6] 预测结果回溯合并与保存 ---
✅ 最终结果文件已成功保存到: c:\Users\yangj\Desktop\4JYY\4JYY\Transfer Learning\SVM\checkpoints\results\tgt_data_with_predictions.csv
文件已包含所有原始列，并在末尾附加了 'predicted_multipath' 列。
